In [1]:
from evalml.pipelines import BinaryClassificationPipeline
from sklearn import datasets
import time
from evalml.tests.automl_tests.test_dask_engine import test_automl_data
from evalml.automl.engine.dask_engine import DaskEngine
from evalml.automl.engine.sequential_engine import SequentialEngine
from dask.distributed import Client, progress
import pandas as pd

/Users/karsten.chu/.pyenv/versions/3.8.0/envs/dev/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# Get some information about the local cluster that's made
client = Client()
client.close()

client

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54781 instead


Client Scheduler: tcp://127.0.0.1:54782 Dashboard: http://127.0.0.1:54781/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
def do_parallel_1(func, pipelines, X, y):
    """ A method of performing pipeline evaluation in parallel"""
    futures = []
    for pipeline in pipelines:
        futures.append(func(automl_data=automl_data, pipeline=pipeline,X=X, y=y))
    results = [f.get_result() for f in futures]
    return results

def time_parallel(func, pipelines, X, y, client):
    parallel_time_start = time.time()
    X, y = client.scatter([X,y])
    do_parallel_1(func, pipelines, X, y)
    parallel_time_taken = time.time()-parallel_time_start
    return parallel_time_taken, results

In [7]:
def time_sequential(func, pipelines, X, y):
    sequential_time_start = time.time()
    futures = []
    for pipeline in pipelines:
        futures.append(func(X=X, y=y, automl_data=automl_data, pipeline=pipeline))
    results = [f.get_result() for f in futures]
    sequential_time_taken = time.time()-sequential_time_start
    return sequential_time_taken, results                                                                      

In [9]:
automl_data = test_automl_data()

# Instantiate client
client = Client()

class TestBinaryClassificationPipeline(BinaryClassificationPipeline):
    component_graph = ["Logistic Regression Classifier"]
    
num_pipelines = 16
pipelines = []
for i in range(num_pipelines):
    pipelines.append(TestBinaryClassificationPipeline({}))
    
seq_times = []
par_times = []
speed_up = []
num_samples= [10, 40, 160, 640, 2560, 5120, 10240, 20480, 40960, 81920]

for n_s in num_samples:
    X, y = datasets.make_classification(n_samples=n_s, n_features=200,
                                    n_informative=2, n_redundant=2, random_state=0)

    engine = DaskEngine(client)
    parallel_time_start = time.time()
    X, y = client.scatter([X,y])
    futures = []
    for pipeline in pipelines:
        futures.append(engine.submit_training_job(automl_data=automl_data, pipeline=pipeline, X=X, y=y))
    results = [f.get_result() for f in futures]
    parallel_time_taken = time.time()-parallel_time_start
    par_times.append(parallel_time)

    engine = SequentialEngine()
    sequential_time_start = time.time()
    futures = []
    for pipeline in pipelines:
        futures.append(engine.submit_training_job(X=X, y=y, automl_data=automl_data, pipeline=pipeline))
    results = [f.get_result() for f in futures]
    sequential_time_taken = time.time()-sequential_time_start
    seq_times.append(sequential_time)
    speed_up.append(sequential_time/parallel_time)
    
    print("Num_samples:", n_s, "Sequential Time: ", sequential_time, "Parallel Time:", parallel_time, 
          "Speedup:", speed_up[-1])
    
# Close client
client.close()

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54908 instead


AttributeError: 'Future' object has no attribute 'copy'

In [ ]:
df = pd.DataFrame()
df["Num Samples"] = num_samples
df["Sequential Time"] = seq_times
df["Parallel Time"] = par_times

ax = df.plot(kind="scatter", x="Num Samples", y = "Sequential Time", color="r")
df.plot(kind="scatter", x="Num Samples", y = "Parallel Time", color="g", ax=ax)
ax.legend(["Sequential", "Parallel"])
ax.set_title("Training Pipelines")
ax.set_xlabel("Num Samples")
ax.set_ylabel("Wall Clock Time (s)")

In [ ]:
for n_s in num_samples:
    X, y = datasets.make_classification(n_samples=n_s, n_features=200,
                                    n_informative=2, n_redundant=2, random_state=0)

    engine = DaskEngine(client)
    parallel_time, parallel_results = time_parallel(engine.submit_training_job, pipelines, X, y, client)
    par_times.append(parallel_time)

    engine = SequentialEngine()
    sequential_time, sequential_results = time_sequential(engine.submit_training_job, pipelines, X, y)
    seq_times.append(sequential_time)
    speed_up.append(sequential_time/parallel_time)
    
    print("Num_samples:", n_s, "Sequential Time: ", sequential_time, "Parallel Time:", parallel_time, 
          "Speedup:", speed_up[-1])
    

In [ ]:
# without scatter Sequential Time:  5.393399953842163 Parallel Time: 3.6696488857269287

# 40000 samples, 200 n_features
# Without scatter Sequential Time:  27.692340850830078 Parallel Time: 11.069131135940552
# With scatter/broadcast = False Sequential Time: 27.925297737121582 Parallel Time: 11.293490886688232
# With scatter/broadcast = True Sequential Time:  27.503095149993896 Parallel Time: 11.242128133773804

# 40000 samples, 2000 n_features, n_pipelines = 16
# Without scatter Sequential Time:  122.6994891166687 Parallel Time: 119.01962494850159
# With scatter/broadcast = False Sequential Time:  123.0830340385437 Parallel Time: 90.91308689117432
# With scatter/broadcast = True Sequential Time: 123.36094689369202 Parallel Time: 102.01213908195496



In [ ]:
\\\\\\\\\